In [1]:
import tifffile
import numpy as np
import os
from collections import defaultdict
import sys
import re
import h5py

In [2]:
name_regex = re.compile(r"""[\w-]+Pos_(\d+)_(\d+)""")
image_directory = 'Sm_Movies/'
name = image_directory

In [3]:
#def get_all_tif_paths(image_directory):
paths = defaultdict(set)
for directory, subdirs, filenames in os.walk(image_directory):
    if not filenames:
        continue
    for filename in filenames:
        if not filename.endswith('ome.tif'):
            continue
        paths[directory].add(os.path.join(directory, filename))

In [4]:
tif_stack = []
for directory, tifs in paths.items():
    tif_stack += list(tifs)
print tif_stack

['Sm_Movies/Cy5/Cy5_Pos_7_02.ome.tif', 'Sm_Movies/Cy5/Cy5_Pos_5_01.ome.tif', 'Sm_Movies/Cy5/Cy5_Pos_7_01.ome.tif', 'Sm_Movies/Cy5/Cy5_Pos_3_02.ome.tif', 'Sm_Movies/Cy5/Cy5_Pos_4_02.ome.tif', 'Sm_Movies/Cy5/Cy5_Pos_8_02.ome.tif', 'Sm_Movies/Cy5/Cy5_Pos_6_01.ome.tif', 'Sm_Movies/Cy5/Cy5_Pos_6_02.ome.tif', 'Sm_Movies/Cy5/Cy5_Pos_2_01.ome.tif', 'Sm_Movies/Cy5/Cy5_Pos_1_02.ome.tif', 'Sm_Movies/Cy5/Cy5_Pos_9_01.ome.tif', 'Sm_Movies/Cy5/Cy5_Pos_0_01.ome.tif', 'Sm_Movies/Cy5/Cy5_Pos_1_01.ome.tif', 'Sm_Movies/Cy5/Cy5_Pos_9_02.ome.tif', 'Sm_Movies/Cy5/Cy5_Pos_3_01.ome.tif', 'Sm_Movies/Cy5/Cy5_Pos_0_02.ome.tif', 'Sm_Movies/Cy5/Cy5_Pos_5_02.ome.tif', 'Sm_Movies/Cy5/Cy5_Pos_8_01.ome.tif', 'Sm_Movies/Cy5/Cy5_Pos_4_01.ome.tif', 'Sm_Movies/Cy5/Cy5_Pos_2_02.ome.tif', 'Sm_Movies/Cy3/Cy3_Pos_7_02.ome.tif', 'Sm_Movies/Cy3/Cy3_Pos_5_01.ome.tif', 'Sm_Movies/Cy3/Cy3_Pos_7_01.ome.tif', 'Sm_Movies/Cy3/Cy3_Pos_3_02.ome.tif', 'Sm_Movies/Cy3/Cy3_Pos_4_02.ome.tif', 'Sm_Movies/Cy3/Cy3_Pos_8_02.ome.tif', 'Sm_Movies/

In [5]:
#def get_tif_positions(tif_stack):
tif_axes = {}
best_first = 0
best_second = 0
for file_path in tif_stack:
    filename = os.path.split(file_path)[1]
    axis_positions = name_regex.search(filename)
    first = int(axis_positions.group(1))
    second = int(axis_positions.group(2))
    best_first = max(first, best_first)
    best_second = max(second, best_second)
    tif_axes[file_path] = (first, second)
if best_second > best_first:
    # the second thing is the major axis, so we need to invert them
    self._axes = {file_path: (second, first) for file_path, (first, second) in tif_axes.items()}
print tif_axes

{'Sm_Movies/Cy5/Cy5_Pos_7_02.ome.tif': (7, 2), 'Sm_Movies/Cy5/Cy5_Pos_5_01.ome.tif': (5, 1), 'Sm_Movies/Cy3/Cy3_Pos_4_02.ome.tif': (4, 2), 'Sm_Movies/Cy3/Cy3_Pos_6_01.ome.tif': (6, 1), 'Sm_Movies/Cy5/Cy5_Pos_6_02.ome.tif': (6, 2), 'Sm_Movies/Cy5/Cy5_Pos_2_01.ome.tif': (2, 1), 'Sm_Movies/Cy3/Cy3_Pos_1_02.ome.tif': (1, 2), 'Sm_Movies/Cy5/Cy5_Pos_0_02.ome.tif': (0, 2), 'Sm_Movies/Cy5/Cy5_Pos_5_02.ome.tif': (5, 2), 'Sm_Movies/Cy5/Cy5_Pos_9_01.ome.tif': (9, 1), 'Sm_Movies/Cy5/Cy5_Pos_4_01.ome.tif': (4, 1), 'Sm_Movies/Cy3/Cy3_Pos_8_02.ome.tif': (8, 2), 'Sm_Movies/Cy3/Cy3_Pos_3_02.ome.tif': (3, 2), 'Sm_Movies/Cy5/Cy5_Pos_4_02.ome.tif': (4, 2), 'Sm_Movies/Cy5/Cy5_Pos_6_01.ome.tif': (6, 1), 'Sm_Movies/Cy5/Cy5_Pos_1_02.ome.tif': (1, 2), 'Sm_Movies/Cy3/Cy3_Pos_0_01.ome.tif': (0, 1), 'Sm_Movies/Cy3/Cy3_Pos_9_02.ome.tif': (9, 2), 'Sm_Movies/Cy3/Cy3_Pos_3_01.ome.tif': (3, 1), 'Sm_Movies/Cy3/Cy3_Pos_8_01.ome.tif': (8, 1), 'Sm_Movies/Cy5/Cy5_Pos_8_02.ome.tif': (8, 2), 'Sm_Movies/Cy3/Cy3_Pos_7_01.ome.t

In [6]:
#def create_h5_dictionary(positions):
h5_entries = defaultdict(lambda: defaultdict(int))
for file_name, position in tif_axes.items():
    if 'Cy3' in file_name:
        major_position = str(position[0])
        minor_position = str(position[1])
        positions = '(Major, minor) = ({}, {})'.format(major_position, minor_position)
        print positions
        with tifffile.TiffFile(file_name) as tif:
            img = tif.asarray()
            sum_img = np.sum(img, dtype=np.int, axis=0)
            print sum_img.size, sum_img.shape
            h5_entries['Cy3'][positions] = sum_img
    if 'Cy5' in file_name:
        print file_name
        major_position = str(position[0])
        minor_position = str(position[1])
        positions = '(Major, minor) = ({}, {})'.format(major_position, minor_position)
        print positions
        with tifffile.TiffFile(file_name) as tif:
            image = tif.asarray()
            #for adjustment in self._adjustments:
            #    image = adjustment(image)
            sum_image = np.sum(image, dtype=np.uint8, axis=0)
            print sum_image.size, sum_image.shape
            h5_entries['Cy5'][positions] = sum_image
return h5_entries

Sm_Movies/Cy5/Cy5_Pos_7_02.ome.tif
(Major, minor) = (7, 2)
262144 (512, 512)
Sm_Movies/Cy5/Cy5_Pos_5_01.ome.tif
(Major, minor) = (5, 1)
262144 (512, 512)
(Major, minor) = (4, 2)
262144 (512, 512)
(Major, minor) = (6, 1)
262144 (512, 512)
Sm_Movies/Cy5/Cy5_Pos_6_02.ome.tif
(Major, minor) = (6, 2)
262144 (512, 512)
Sm_Movies/Cy5/Cy5_Pos_2_01.ome.tif
(Major, minor) = (2, 1)
262144 (512, 512)
(Major, minor) = (1, 2)
262144 (512, 512)
Sm_Movies/Cy5/Cy5_Pos_0_02.ome.tif
(Major, minor) = (0, 2)
262144 (512, 512)
Sm_Movies/Cy5/Cy5_Pos_5_02.ome.tif
(Major, minor) = (5, 2)
262144 (512, 512)
Sm_Movies/Cy5/Cy5_Pos_9_01.ome.tif
(Major, minor) = (9, 1)
262144 (512, 512)
Sm_Movies/Cy5/Cy5_Pos_4_01.ome.tif
(Major, minor) = (4, 1)
262144 (512, 512)
(Major, minor) = (8, 2)
262144 (512, 512)
(Major, minor) = (3, 2)
262144 (512, 512)
Sm_Movies/Cy5/Cy5_Pos_4_02.ome.tif
(Major, minor) = (4, 2)
262144 (512, 512)
Sm_Movies/Cy5/Cy5_Pos_6_01.ome.tif
(Major, minor) = (6, 1)
262144 (512, 512)
Sm_Movies/Cy5/Cy5_Po

SyntaxError: 'return' outside function (<ipython-input-6-1f206e2f1082>, line 27)

In [7]:
#def convert_image_dict_to_h5(name, datadict):
#    """Convert nested dictionary to h5 file
#   
#    Parameters
#    ----------
#    name: str
#        Name of the movie; used to name h5 file
#    
#    datadict: dict
#        Dictionary mapping channel names and positions to pixel intensity arrays
#
#    """
    
h = h5py.File('{}.h5'.format(name), 'w')
for ch_name in h5_entries.keys():
    print ch_name
    for pos in h5_entries[ch_name].keys():
        print pos
        h.create_dataset('{}/{}'.format(ch_name, pos), data=h5_entries[ch_name][pos])
h.close()

Cy3
(Major, minor) = (7, 1)
(Major, minor) = (3, 2)
(Major, minor) = (1, 2)
(Major, minor) = (2, 2)
(Major, minor) = (7, 2)
(Major, minor) = (0, 2)
(Major, minor) = (8, 1)
(Major, minor) = (6, 2)
(Major, minor) = (2, 1)
(Major, minor) = (4, 2)
(Major, minor) = (9, 2)
(Major, minor) = (6, 1)
(Major, minor) = (8, 2)
(Major, minor) = (9, 1)
(Major, minor) = (4, 1)
(Major, minor) = (5, 1)
(Major, minor) = (5, 2)
(Major, minor) = (3, 1)
(Major, minor) = (1, 1)
(Major, minor) = (0, 1)
Cy5
(Major, minor) = (7, 1)
(Major, minor) = (3, 2)
(Major, minor) = (4, 1)
(Major, minor) = (2, 2)
(Major, minor) = (7, 2)
(Major, minor) = (0, 2)
(Major, minor) = (5, 2)
(Major, minor) = (6, 2)
(Major, minor) = (2, 1)
(Major, minor) = (4, 2)
(Major, minor) = (9, 2)
(Major, minor) = (6, 1)
(Major, minor) = (5, 1)
(Major, minor) = (9, 1)
(Major, minor) = (1, 2)
(Major, minor) = (8, 2)
(Major, minor) = (8, 1)
(Major, minor) = (3, 1)
(Major, minor) = (1, 1)
(Major, minor) = (0, 1)
